In [1]:
#Testing the semtui package 
import semtui_refactored

In [2]:
# Import necessary classes and functions from the semtui_refactored package
from semtui_refactored.data_handler import DataHandler
from semtui_refactored.token_manager import TokenManager
from semtui_refactored.extension_manager import ExtensionManager
from semtui_refactored.reconciliation_manager import ReconciliationManager
from semtui_refactored.utils import Utility
from semtui_refactored.dataset_manager import DatasetManager
from semtui_refactored.semtui_evals import EvaluationManager
from semtui_refactored.data_modifier import DataModifier


# Set up the API URL and credentials
api_url = "http://localhost:3003/api/"  # The base URL for the API
username = "test"  # Username for authentication
password = "test"  # Password for authentication

# Initialize TokenManager
signin_data = {"username": username, "password": password}  # Payload for sign-in request
signin_headers = {
    "accept": "application/json",  # Specify the response format
    "content-type": "application/json"  # Specify the request content type
}
token_manager = TokenManager(api_url, signin_data, signin_headers)  # Create an instance of TokenManager

# Get token
token = token_manager.get_token()  # Retrieve the authentication token
#print(f"Token: {token}")  # Uncomment to print the token (useful for debugging)


In [3]:
# Initialize ReconciliationManager
reconciliation_manager = ReconciliationManager(api_url, token_manager)  # Create an instance of ReconciliationManager with API URL and token manager

# Initialize DatasetManager
dataset_manager = DatasetManager(api_url, token_manager)  # Create an instance of DatasetManager with API URL and token manager

# Initialize the EvaluationManager
evaluation_manager = EvaluationManager()  # Create an instance of EvaluationManager

# Initialize ExtensionManager
extension_manager = ExtensionManager(api_url, token)  # Create an instance of ExtensionManager with API URL and token

# Initialize the DataModifier
data_modifier_manager = DataModifier()  # Create an instance of EvaluationManager


In [4]:
import os 
import json 
import pandas as pd
import numpy as np
import requests

In [5]:
# Configure Pandas Display Options

#This segment of code configures the display options for Pandas DataFrames. 
#Adjusting these settings allows for better visibility and control over how DataFrames are presented in the notebook.

### Code Explanation
#- Import the Pandas library.
#- Set the display option to show all columns of a DataFrame.
#- Limit the display to show only the first 20 rows of a DataFrame.
    
import pandas as pd 
# Set pandas display options

pd.set_option('display.max_columns', None)  # Display all columns
pd.set_option('display.max_rows', 20)  # Limit to 20 rows for display

In [6]:

# Get the current working directory
cwd = os.getcwd()
print("Current working directory:", cwd)

# Path to your CSV file
csv_file_path = "/Users/abubakarialidu/Documents/Semtui_test/JOT data tutorial notebook tiny.csv"  # Define the path to the CSV file

# Read CSV data using DataManager
try:
    df = pd.read_csv(csv_file_path)  # Read the CSV file into a DataFrame using DataManager
    print("CSV file imported successfully!")  # Print success message
    display(df.head())  # Display the first few rows of the DataFrame
except Exception as e:
    print(f"Error importing CSV file: {e}")  # Print error message if CSV import fails


Current working directory: /Users/abubakarialidu/Documents/SEMT-py/semtui1.1
CSV file imported successfully!


,Fecha_id,Keyword,Impresiones,Clicks,City,County,Country
0,20230101,alquiler pisos colindres,1,0,Madrid,Community of Madrid,Spain
1,20230101,alquiler pisos sestao,1,0,Barcelona,Catalonia,Spain
2,20230101,steelcraft pedal car,1,0,Buffalo,New York,United States


In [7]:
# Use DataModifier to modify the DataFrame
try:
    # Convert the 'Fecha_id' column to ISO date format
    df = DataModifier.iso_date(df, date_col='Fecha_id')

    # Uncomment the following lines if you need to perform the respective operations

    # df = DataModifier.lower_case(df, column='column_name')  # Convert column values to lowercase
    # df = DataModifier.drop_na(df)  # Remove rows with missing values
    # df = DataModifier.rename_columns(df, column_rename_dict={'old_name': 'new_name'})  # Rename columns
    # df = DataModifier.convert_dtypes(df, dtype_dict={'column_name': 'int'})  # Convert column data types
    # df = DataModifier.reorder_columns(df, new_column_order=['col1', 'col2', 'col3'])  # Reorder columns

    print("DataFrame modification successful!")
    
except ValueError as e:
    print(e)

DataFrame modification successful!


In [8]:
dataset_id = '30'
table_name = 'test_e2'

In [9]:
try:
    dataset_manager.add_table_to_dataset(dataset_id, df, table_name)  # Attempt to add the DataFrame as a table to the dataset
    print(f"Table '{table_name}' added to dataset ID {dataset_id} successfully.")  # Print success message
except Exception as e:
    print(f"Error adding table to dataset: {e}")  # Print error message if adding table fails


Table added successfully!
New table added: ID: 160, Name: test_e2
Table 'test_e2' added to dataset ID 30 successfully.


In [10]:
Table_id = '160'

In [11]:
try:
    table_data = dataset_manager.get_table_by_name(dataset_id, table_name)  # Attempt to retrieve the specified table from the dataset
    if table_data:
        print(f"Table '{table_name}' retrieved successfully!")  # Print success message if table is retrieved
        # No need to display the DataFrame
    else:
        print(f"Table '{table_name}' not found in the dataset.")  # Print message if table is not found
except Exception as e:
    print(f"Error retrieving table '{table_name}': {e}")  # Print error message if retrieving the table fails

Table 'test_e2' retrieved successfully!


In [12]:
#table_data

### Reconciliation  

In [40]:
# Load JSON files from drive
def load_json_from_drive(file_path):
    import json
    with open(file_path, 'r') as file:
        return json.load(file)

reconciled_json_path = '/Users/abubakarialidu/Documents/SEMT-py/semtui1.1/semtui_refactored/my_reconciled_table.json'
reconciled_json = load_json_from_drive(reconciled_json_path)
#reconciled_json

In [98]:
#reconciled_json

In [41]:
def create_reconciliation_payload_for_backend(table_json, reconciliated_column_name, table_id, Dataset_id, table_name):
    """
    Creates the payload required to perform the table update operation.
    
    :param table_json: JSON representation of the table
    :param reconciliated_column_name: The name of the column that contains the reconciliated information
    :param table_id: ID of the table
    :param Dataset_id: ID of the dataset
    :param table_name: Name of the table
    :return: request payload
    """
    # Default values for missing fields
    default_table_fields = {
        "minMetaScore": 0,
        "maxMetaScore": 1
    }

    # Ensure all required fields are present in the table section
    table_data = {**default_table_fields, **table_json.get("table", {})}

    # Recalculate nCellsReconciliated
    nCellsReconciliated = 0

    # Check the specified reconciliated column for reconciliated count
    if reconciliated_column_name in table_json["columns"]:
        column = table_json["columns"][reconciliated_column_name]
     if "context" in column and "georss" in column["context"]:
            nCellsReconciliated = column["context"]["georss"].get("reconciliated", 0)
    
    # Override id and idDataset with provided values
    table_data["id"] = table_id
    table_data["idDataset"] = Dataset_id
    table_data["name"] = table_name

    # Construct the payload
    payload = {
        "tableInstance": {
            "id": table_data["id"],
            "idDataset": table_data["idDataset"],
            "name": table_data["name"],
            "nCols": table_data["nCols"],
            "nRows": table_data["nRows"],
            "nCells": table_data["nCells"],
            "nCellsReconciliated": nCellsReconciliated,
            "lastModifiedDate": table_data["lastModifiedDate"],
            "minMetaScore": table_data["minMetaScore"],
            "maxMetaScore": table_data["maxMetaScore"]
        },
        "columns": {
            "byId": table_json["columns"],
            "allIds": list(table_json["columns"].keys())
        },
        "rows": {
            "byId": table_json["rows"],
            "allIds": list(table_json["rows"].keys())
        }
    }
    return payload

In [42]:
reconciliated_column_name = 'City'
table_id = '160'
Dataset_id = '30'
table_name = 'test_e2'
reconciled_json_payload = create_reconciliation_payload_for_backend(reconciled_json, reconciliated_column_name, table_id, Dataset_id, table_name)

In [43]:
reconciled_json_payload

{'tableInstance': {'id': '160',
  'idDataset': '30',
  'name': 'test_e2',
  'nCols': 7,
  'nRows': 3,
  'nCells': 21,
  'nCellsReconciliated': 3,
  'lastModifiedDate': '2024-06-07T14:05:32.939Z',
  'minMetaScore': 0,
  'maxMetaScore': 1},
 'columns': {'byId': {'Fecha_id': {'id': 'Fecha_id',
    'label': 'Fecha_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Keyword': {'id': 'Keyword',
    'label': 'Keyword',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Impresiones': {'id': 'Impresiones',
    'label': 'Impresiones',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Clicks': {'id': 'Clicks',
    'label': 'Clicks',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'City': {'id': 'City',
    'label': 'City',
    'status': 'pending',
    'context': {'georss': {'uri': 'http://www.google.com/maps/place/',
      'total': 3,
      'reconciliated': 3}},
    'metadata': [{'id': '',
      'match': True,
      'score': 0

In [44]:
def push_reconciliation_data_to_backend(token_manager, dataset_id, table_id, Reconciled_data, api_url):
    """
    Pushes reconciliation data to the backend.

    :param token_manager: Instance of TokenManager to handle authentication tokens
    :param dataset_id: ID of the dataset
    :param table_id: ID of the table
    :param Reconciled_data: Reconciled data to be sent
    :param api_url: Base URL of the API
    :return: Response from the backend
    """
    # Get the token
    token = token_manager.get_token()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Create the endpoint URL
    url = f"{api_url}dataset/{dataset_id}/table/{table_id}"

    # Create the update payload
    update_payload = reconciled_json_payload

    try:
        # Send the PUT request to update the table
        response = requests.put(url, headers=headers, json=update_payload)
        
        if response.status_code == 200:
            print("Table updated successfully!")
            return response.json()
        elif response.status_code == 401:
            print("Unauthorized: Invalid or missing token.")
        elif response.status_code == 404:
            print(f"Dataset or table with ID {dataset_id}/{table_id} not found.")
        else:
            print(f"Failed to update table: {response.status_code}, {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while updating table: {e}")
        return None

In [45]:
push_reconciliation_data_to_backend(token_manager, dataset_id, table_id, reconciled_json_payload, api_url)

Table updated successfully!


{'id': '160',
 'idDataset': '30',
 'name': 'test_e2',
 'nCols': 7,
 'nRows': 3,
 'nCells': 21,
 'nCellsReconciliated': 3,
 'lastModifiedDate': '2024-06-20T14:00:46.155Z',
 'minMetaScore': 0,
 'maxMetaScore': 1}

## Extension 1

NameError: name 'load_json_from_drive' is not defined

In [26]:
def normalize_json_structure(json_data):
    """
    Normalize the JSON structure to have 'byId' and 'allIds' for columns and rows.
    """
    if 'columns' in json_data and 'byId' not in json_data['columns']:
        columns_by_id = {col_id: col for col_id, col in json_data['columns'].items()}
        columns_all_ids = list(columns_by_id.keys())
        json_data['columns'] = {'byId': columns_by_id, 'allIds': columns_all_ids}

    if 'rows' in json_data and 'byId' not in json_data['rows']:
        rows_by_id = {row_id: row for row_id, row in json_data['rows'].items()}
        rows_all_ids = list(rows_by_id.keys())
        json_data['rows'] = {'byId': rows_by_id, 'allIds': rows_all_ids}

    return json_data

def merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties):
    reconciled_json = normalize_json_structure(reconciled_json)
    extension_json = normalize_json_structure(extension_json)
    
    merged_json = extension_json.copy()
    additional_columns = [f"{reconciliated_column_name}_{prop}" for prop in properties]

    # Merge columns
    for col_id, col in reconciled_json['columns']['byId'].items():
        if col_id in merged_json['columns']['byId']:
            if col_id == reconciliated_column_name:
                merged_json['columns']['byId'][col_id]['annotationMeta'] = col.get('annotationMeta', {})
                merged_json['columns']['byId'][col_id]['metadata'] = [
                    {k: v for k, v in meta.items() if k != 'entity' and k != 'property' and k != 'type'}
                    for meta in merged_json['columns']['byId'][col_id]['metadata']
                ]
                merged_json['columns']['byId'][col_id]['status'] = 'reconciliated'
                for meta in merged_json['columns']['byId'][col_id]['metadata']:
                    meta['entity'] = col['metadata'][0].get('entity', [])
                merged_json['columns']['byId'][col_id]['annotationMeta']['match']['reason'] = 'reconciliator'
            else:
                merged_json['columns']['byId'][col_id] = col
        else:
            merged_json['columns']['byId'][col_id] = col

    # Remove the additional columns based on properties
    columns_to_remove = [f"id_{reconciliated_column_name}", f"name_{reconciliated_column_name}"]
    merged_json['columns']['byId'] = {
        col_id: col for col_id, col in merged_json['columns']['byId'].items()
        if col_id not in columns_to_remove
    }

    # Add the additional columns for weather data
    for col_id in additional_columns:
        if col_id not in merged_json['columns']['byId']:
            merged_json['columns']['byId'][col_id] = {
                'id': col_id,
                'label': col_id,
                'status': 'empty',
                'context': {},
                'metadata': []
            }
            merged_json['columns']['allIds'].append(col_id)

    # Merge rows
    for row_id, row in reconciled_json['rows']['byId'].items():
        if row_id in merged_json['rows']['byId']:
            for cell_id, cell in row['cells'].items():
                if cell_id in merged_json['rows']['byId'][row_id]['cells']:
                    if cell_id == reconciliated_column_name:
                        merged_json['rows']['byId'][row_id]['cells'][cell_id]['annotationMeta'] = cell.get('annotationMeta', {})
                        merged_json['rows']['byId'][row_id]['cells'][cell_id]['annotationMeta']['match']['reason'] = 'reconciliator'
                    else:
                        merged_json['rows']['byId'][row_id]['cells'][cell_id] = cell
                else:
                    merged_json['rows']['byId'][row_id]['cells'][cell_id] = cell
        else:
            merged_json['rows']['byId'][row_id] = row
            merged_json['rows']['allIds'].append(row_id)

    # Ensure the number of cells matches
    for row_id, row in merged_json['rows']['byId'].items():
        backend_cells = set(reconciled_json['rows']['byId'][row_id]['cells'].keys())
        extension_cells = set(extension_json['rows']['byId'][row_id]['cells'].keys())
        missing_cells = backend_cells - extension_cells

        for cell_id in missing_cells:
            merged_json['rows']['byId'][row_id]['cells'][cell_id] = reconciled_json['rows']['byId'][row_id]['cells'][cell_id]

    # Remove id_City and name_City from rows
    for row_id, row in merged_json['rows']['byId'].items():
        cells_to_remove = [f"id_{reconciliated_column_name}", f"name_{reconciliated_column_name}"]
        merged_json['rows']['byId'][row_id]['cells'] = {
            cell_id: cell for cell_id, cell in row['cells'].items()
            if cell_id not in cells_to_remove
        }

    # Update table metadata
    merged_json['table']['nCols'] = len(merged_json['columns']['byId'])
    merged_json['table']['nCells'] = sum(len(row['cells']) for row in merged_json['rows']['byId'].values())
    merged_json['table']['nCellsReconciliated'] = sum(
        1 for row in merged_json['rows']['byId'].values() for cell in row['cells'].values() if cell.get('annotationMeta', {}).get('annotated', False)
    )
    merged_json['table']['maxMetaScore'] = max(
        (cell.get('annotationMeta', {}).get('highestScore', 0) for row in merged_json['rows']['byId'].values() for cell in row['cells'].values()), default=0
    )

    return merged_json

def format_json_for_backend_with_byId(merged_json, reconciliated_column_name):
    backend_json = {
        "tableInstance": {
            "id": merged_json["table"]["id"],
            "idDataset": merged_json["table"]["idDataset"],
            "name": merged_json["table"]["name"],
            "nCols": merged_json["table"]["nCols"],
            "nRows": merged_json["table"]["nRows"],
            "nCells": merged_json["table"]["nCells"],
            "nCellsReconciliated": merged_json["table"]["nCellsReconciliated"],
            "lastModifiedDate": merged_json["table"]["lastModifiedDate"],
            "minMetaScore": merged_json["table"].get("minMetaScore", 0),
            "maxMetaScore": merged_json["table"].get("maxMetaScore", 0)
        },
        "columns": {
            "byId": {},
            "allIds": []
        },
        "rows": {
            "byId": {},
            "allIds": []
        },
        "id": merged_json["id"]
    }

    # Process columns
    for col_id, col in merged_json['columns']['byId'].items():
        backend_json['columns']['byId'][col_id] = {
            'id': col['id'],
            'label': col['label'],
            'status': col['status'],
            'context': col['context'],
            'metadata': [
                {
                    'id': meta.get('id', ''),
                    'match': meta.get('match', False),
                    'score': meta.get('score', 0),
                    'name': meta.get('name', {}),
                    'entity': meta.get('entity', [])
                }
                for meta in col.get('metadata', [])
            ]
        }
        # Only add annotationMeta for the reconciliated column
        if col_id == reconciliated_column_name:
            backend_json['columns']['byId'][col_id]['annotationMeta'] = col.get('annotationMeta', {})
            backend_json['columns']['byId'][col_id]['annotationMeta']['match']['reason'] = 'reconciliator'
        # Remove property from metadata
        for meta in backend_json['columns']['byId'][col_id]['metadata']:
            meta.pop('property', None)
            if col_id == reconciliated_column_name:
                meta['type'] = []  # Set type to empty list for the column metadata

        backend_json['columns']['allIds'].append(col_id)

    # Process rows
    for row_id, row in merged_json['rows']['byId'].items():
        backend_json['rows']['byId'][row_id] = {
            'id': row['id'],
            'cells': {}
        }
        for cell_id, cell in row['cells'].items():
            backend_json['rows']['byId'][row_id]['cells'][cell_id] = {
                'id': cell['id'],
                'label': cell['label'],
                'metadata': cell.get('metadata', [])
            }
            # Only add annotationMeta for the reconciliated column
            if cell_id == reconciliated_column_name:
                backend_json['rows']['byId'][row_id]['cells'][cell_id]['annotationMeta'] = cell.get('annotationMeta', {})
                backend_json['rows']['byId'][row_id]['cells'][cell_id]['annotationMeta']['match']['reason'] = 'reconciliator'
            # Remove property from metadata
            for meta in backend_json['rows']['byId'][row_id]['cells'][cell_id]['metadata']:
                meta.pop('property', None)

        backend_json['rows']['allIds'].append(row_id)

    return backend_json

In [27]:
#extension_json

{'table': {'id': '148',
  'idDataset': '30',
  'name': 'Jot_test1',
  'nCols': 7,
  'nRows': 3,
  'nCells': 21,
  'nCellsReconciliated': 0,
  'lastModifiedDate': '2024-06-07T14:05:32.939Z'},
 'columns': {'Fecha_id': {'id': 'Fecha_id',
   'label': 'Fecha_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Keyword': {'id': 'Keyword',
   'label': 'Keyword',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Impresiones': {'id': 'Impresiones',
   'label': 'Impresiones',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Clicks': {'id': 'Clicks',
   'label': 'Clicks',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'City': {'id': 'City',
   'label': 'City',
   'status': 'pending',
   'context': {'georss': {'uri': 'http://www.google.com/maps/place/',
     'total': 3,
     'reconciliated': 3}},
   'metadata': [{'id': '',
     'match': True,
     'score': 0,
     'name': {'value': '', 'uri': ''},
     'entity': [{'id': 'wd:Q29934236',
     

In [28]:
# Example usage
properties = ["apparent_temperature_max", "apparent_temperature_min", "precipitation_sum"]  # Replace with actual properties to extend

formatted_json = process_and_format_json(reconciled_json_payload, extension_json, reconciliated_column_name, properties, table_id, idDataset, name)
formatted_json

{'tableInstance': {'id': '153',
  'idDataset': '30',
  'name': 'JOT_extension_1',
  'nCols': 10,
  'nRows': 3,
  'nCells': 30,
  'nCellsReconciliated': 3,
  'lastModifiedDate': '2024-06-07T14:05:32.939Z',
  'minMetaScore': 0,
  'maxMetaScore': 1},
 'columns': {'byId': {'Fecha_id': {'id': 'Fecha_id',
    'label': 'Fecha_id',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Keyword': {'id': 'Keyword',
    'label': 'Keyword',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Impresiones': {'id': 'Impresiones',
    'label': 'Impresiones',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'Clicks': {'id': 'Clicks',
    'label': 'Clicks',
    'status': 'empty',
    'context': {},
    'metadata': []},
   'City': {'id': 'City',
    'label': 'City',
    'status': 'reconciliated',
    'context': {'georss': {'uri': 'http://www.google.com/maps/place/',
      'total': 3,
      'reconciliated': 3}},
    'metadata': [{'id': '',
      'match': True,
 

In [29]:
def push_reconciliation_data_to_backend(token_manager, dataset_id, table_id, formatted_json, api_url):
    """
    Pushes reconciliation data to the backend.

    :param token_manager: Instance of TokenManager to handle authentication tokens
    :param dataset_id: ID of the dataset
    :param table_id: ID of the table
    :param Reconciled_data: Reconciled data to be sent
    :param api_url: Base URL of the API
    :return: Response from the backend
    """
    # Get the token
    token = token_manager.get_token()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Create the endpoint URL
    url = f"{api_url}dataset/{dataset_id}/table/{table_id}"

    # Create the update payload
    update_payload = formatted_json

    try:
        # Send the PUT request to update the table
        response = requests.put(url, headers=headers, json=update_payload)
        
        if response.status_code == 200:
            print("Table updated successfully!")
            return response.json()
        elif response.status_code == 401:
            print("Unauthorized: Invalid or missing token.")
        elif response.status_code == 404:
            print(f"Dataset or table with ID {dataset_id}/{table_id} not found.")
        else:
            print(f"Failed to update table: {response.status_code}, {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while updating table: {e}")
        return None

In [34]:
push_reconciliation_data_to_backend(token_manager, dataset_id, table_id, formatted_json, api_url)

Failed to update table: 500, {"error":"Cannot destructure property 'id' of 'tableInstance' as it is undefined."}


## Extension 2

In [46]:
try:
    table_data = dataset_manager.get_table_by_name(dataset_id, table_name)  # Attempt to retrieve the specified table from the dataset
    if table_data:
        print(f"Table '{table_name}' retrieved successfully!")  # Print success message if table is retrieved
        # No need to display the DataFrame
    else:
        print(f"Table '{table_name}' not found in the dataset.")  # Print message if table is not found
except Exception as e:
    print(f"Error retrieving table '{table_name}': {e}")  # Print error message if retrieving the table fails

Table 'test_e2' retrieved successfully!


In [54]:
#table_data

In [55]:
# Load JSON files from drive
def load_json_from_drive(file_path):
    import json
    with open(file_path, 'r') as file:
        return json.load(file)

extension_json_path = '/Users/abubakarialidu/Documents/SEMT-py/semtui1.1/semtui_refactored/my_extended_table.json'
extension_json = load_json_from_drive(extension_json_path)

In [99]:
extension_json

{'table': {'id': '148',
  'idDataset': '30',
  'name': 'Jot_test1',
  'nCols': 10,
  'nRows': 3,
  'nCells': 30,
  'nCellsReconciliated': 3,
  'lastModifiedDate': '2024-06-07T14:05:32.939Z',
  'maxMetaScore': 1},
 'columns': {'Fecha_id': {'id': 'Fecha_id',
   'label': 'Fecha_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Keyword': {'id': 'Keyword',
   'label': 'Keyword',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Impresiones': {'id': 'Impresiones',
   'label': 'Impresiones',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Clicks': {'id': 'Clicks',
   'label': 'Clicks',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'City': {'id': 'City',
   'label': 'City',
   'status': 'reconciliated',
   'context': {'georss': {'uri': 'http://www.google.com/maps/place/',
     'total': 3,
     'reconciliated': 3}},
   'metadata': [{'id': '',
     'match': True,
     'score': 0,
     'name': {'value': '', 'uri': ''},
     'type': [{'

In [75]:
def merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties):
    merged_json = extension_json.copy()
    additional_columns = [f"{reconciliated_column_name}_{prop}" for prop in properties]

    # Merge columns
    for col_id, col in reconciled_json['columns'].items():
        if col_id in merged_json['columns']:
            if col_id == reconciliated_column_name:
                merged_json['columns'][col_id]['annotationMeta'] = col.get('annotationMeta', {})
                merged_json['columns'][col_id]['metadata'] = [
                    {k: v for k, v in meta.items() if k != 'entity' and k != 'property'}
                    for meta in merged_json['columns'][col_id]['metadata']
                ]
                merged_json['columns'][col_id]['status'] = 'reconciliated'
                for meta in merged_json['columns'][col_id]['metadata']:
                    meta['entity'] = col['metadata'][0].get('entity', [])
                merged_json['columns'][col_id]['annotationMeta']['match']['reason'] = 'reconciliator'
            else:
                merged_json['columns'][col_id] = col
        else:
            merged_json['columns'][col_id] = col

    # Remove the additional columns based on properties
    columns_to_remove = [f"id_{reconciliated_column_name}", f"name_{reconciliated_column_name}"]
    merged_json['columns'] = {
        col_id: col for col_id, col in merged_json['columns'].items()
        if col_id not in columns_to_remove
    }

    # Add the additional columns for weather data
    for col_id in additional_columns:
        if col_id not in merged_json['columns']:
            merged_json['columns'][col_id] = {
                'id': col_id,
                'label': col_id,
                'status': 'empty',
                'context': {},
                'metadata': []
            }

    # Merge rows
    for row_id, row in reconciled_json['rows'].items():
        if row_id in merged_json['rows']:
            for cell_id, cell in row['cells'].items():
                if cell_id in merged_json['rows'][row_id]['cells']:
                    if cell_id == reconciliated_column_name:
                        merged_json['rows'][row_id]['cells'][cell_id]['annotationMeta'] = cell.get('annotationMeta', {})
                        merged_json['rows'][row_id]['cells'][cell_id]['annotationMeta']['match']['reason'] = 'reconciliator'
                    else:
                        merged_json['rows'][row_id]['cells'][cell_id] = cell
                else:
                    merged_json['rows'][row_id]['cells'][cell_id] = cell
        else:
            merged_json['rows'][row_id] = row

    # Ensure the number of cells matches
    for row_id, row in merged_json['rows'].items():
        backend_cells = set(reconciled_json['rows'][row_id]['cells'].keys())
        extension_cells = set(extension_json['rows'][row_id]['cells'].keys())
        missing_cells = backend_cells - extension_cells

        for cell_id in missing_cells:
            merged_json['rows'][row_id]['cells'][cell_id] = reconciled_json['rows'][row_id]['cells'][cell_id]

    # Remove id_City and name_City from rows
    for row_id, row in merged_json['rows'].items():
        cells_to_remove = [f"id_{reconciliated_column_name}", f"name_{reconciliated_column_name}"]
        merged_json['rows'][row_id]['cells'] = {
            cell_id: cell for cell_id, cell in row['cells'].items()
            if cell_id not in cells_to_remove
        }

    # Update table metadata
    merged_json['table']['nCols'] = len(merged_json['columns'])
    merged_json['table']['nCells'] = sum(len(row['cells']) for row in merged_json['rows'].values())
    merged_json['table']['nCellsReconciliated'] = sum(
        1 for row in merged_json['rows'].values() for cell in row['cells'].values() if cell.get('annotationMeta', {}).get('annotated', False)
    )
    merged_json['table']['maxMetaScore'] = max(
        (cell.get('annotationMeta', {}).get('highestScore', 0) for row in merged_json['rows'].values() for cell in row['cells'].values()), default=0
    )

    return merged_json

def format_json_for_backend(merged_json, reconciliated_column_name):
    backend_json = {
        "table": {
            "id": merged_json["table"]["id"],
            "idDataset": merged_json["table"]["idDataset"],
            "name": merged_json["table"]["name"],
            "nCols": merged_json["table"]["nCols"],
            "nRows": merged_json["table"]["nRows"],
            "nCells": merged_json["table"]["nCells"],
            "nCellsReconciliated": merged_json["table"]["nCellsReconciliated"],
            "lastModifiedDate": merged_json["table"]["lastModifiedDate"],
            "minMetaScore": merged_json["table"].get("minMetaScore", 0),
            "maxMetaScore": merged_json["table"].get("maxMetaScore", 0)
        },
        "columns": {},
        "rows": {},
        "id": merged_json["id"]
    }

    # Process columns
    for col_id, col in merged_json['columns'].items():
        backend_json['columns'][col_id] = {
            'id': col['id'],
            'label': col['label'],
            'status': col['status'],
            'context': col['context'],
            'metadata': [
                {
                    'id': meta.get('id', ''),
                    'match': meta.get('match', False),
                    'score': meta.get('score', 0),
                    'name': meta.get('name', {}),
                    'entity': meta.get('entity', [])
                }
                for meta in col.get('metadata', [])
            ]
        }
        # Only add annotationMeta for the reconciliated column
        if col_id == reconciliated_column_name:
            backend_json['columns'][col_id]['annotationMeta'] = col.get('annotationMeta', {})
            backend_json['columns'][col_id]['annotationMeta']['match']['reason'] = 'reconciliator'
        # Remove property from metadata
        for meta in backend_json['columns'][col_id]['metadata']:
            meta.pop('property', None)

    # Process rows
    for row_id, row in merged_json['rows'].items():
        backend_json['rows'][row_id] = {
            'id': row['id'],
            'cells': {}
        }
        for cell_id, cell in row['cells'].items():
            backend_json['rows'][row_id]['cells'][cell_id] = {
                'id': cell['id'],
                'label': cell['label'],
                'metadata': cell.get('metadata', [])
            }
            # Only add annotationMeta for the reconciliated column
            if cell_id == reconciliated_column_name:
                backend_json['rows'][row_id]['cells'][cell_id]['annotationMeta'] = cell.get('annotationMeta', {})
                backend_json['rows'][row_id]['cells'][cell_id]['annotationMeta']['match']['reason'] = 'reconciliator'
            # Remove property from metadata
            for meta in backend_json['rows'][row_id]['cells'][cell_id]['metadata']:
                meta.pop('property', None)

    return backend_json

def process_and_format_json(reconciled_json, extension_json, reconciliated_column_name, properties):
    merged_json = merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties)
    formatted_json = format_json_for_backend(merged_json, reconciliated_column_name)
    return formatted_json

# Load JSON files from drive
def load_json_from_drive(file_path):
    import json
    with open(file_path, 'r') as file:
        return json.load(file)

reconciled_json_path = '/Users/abubakarialidu/Documents/SEMT-py/semtui1.1/semtui_refactored/my_reconciled_table.json'
extension_json_path = '/Users/abubakarialidu/Documents/SEMT-py/semtui1.1/semtui_refactored/my_extended_table.json'

reconciled_json = load_json_from_drive(reconciled_json_path)
extension_json = load_json_from_drive(extension_json_path)

# Example usage
reconciliated_column_name = 'City'  # Column that contains reconciled IDs
properties = ["apparent_temperature_max", "apparent_temperature_min", "precipitation_sum"]  # Replace with actual properties to extend

formatted_json = process_and_format_json(reconciled_json, extension_json, reconciliated_column_name, properties)
#print()


In [109]:
def update_city_column_metadata(metadata):
    return [
        {
            'id': 'None:',
            'match': True,
            'score': 0,
            'name': {
                'value': '',
                'uri': ''
            },
            'entity': [
                {
                    'id': 'wd:Q29934236',
                    'name': {
                        'value': 'GlobeCoordinate',
                        'uri': 'http://149.132.176.67:3002/map?polyline=Q29934236'
                    },
                    'score': 0,
                    'match': True,
                    'type': []
                },
                {
                    'id': 'georss:point',
                    'name': {
                        'value': 'point',
                        'uri': 'http://149.132.176.67:3002/map?polyline=point'
                    },
                    'score': 0,
                    'match': True,
                    'type': []
                }
            ]
        }
    ]

def update_city_row_metadata(metadata):
    return [
        {
            'id': metadata[0]['id'],
            'name': metadata[0]['name'],
            'feature': [
                {
                    'id': 'all_labels',
                    'value': 100
                }
            ],
            'score': metadata[0]['score'],
            'match': metadata[0]['match'],
            'type': [
                {
                    'id': 'wd:Q29934236',
                    'name': 'GlobeCoordinate'
                },
                {
                    'id': 'georss:point',
                    'name': 'point'
                }
            ]
        }
    ]

def merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties):
    merged_json = {
        'table': extension_json['table'],
        'columns': {},
        'rows': {},
        'id': extension_json['id']
    }

    # Process columns
    for col_id, col in reconciled_json['columns'].items():
        merged_json['columns'][col_id] = {
            'id': col['id'],
            'label': col['label'],
            'status': col['status'],
            'context': col['context'],
            'metadata': col['metadata']
        }
        if col_id == reconciliated_column_name:
            merged_json['columns'][col_id]['status'] = 'reconciliated'
            merged_json['columns'][col_id]['metadata'] = update_city_column_metadata(col['metadata'])
            merged_json['columns'][col_id]['annotationMeta'] = {
                'annotated': True,
                'match': {
                    'value': True,
                    'reason': 'reconciliator'
                },
                'lowestScore': 0,
                'highestScore': 0
            }

    # Add weather columns
    for prop in properties:
        col_id = f"{reconciliated_column_name}_{prop}"
        merged_json['columns'][col_id] = {
            'id': col_id,
            'label': col_id,
            'metadata': [],
            'status': 'empty',
            'context': {}
        }

    # Process rows
    for row_id, row in reconciled_json['rows'].items():
        merged_json['rows'][row_id] = {
            'id': row['id'],
            'cells': {}
        }
        for cell_id, cell in row['cells'].items():
            merged_json['rows'][row_id]['cells'][cell_id] = {
                'id': cell['id'],
                'label': cell['label'],
                'metadata': cell.get('metadata', [])
            }
            if cell_id == reconciliated_column_name:
                merged_json['rows'][row_id]['cells'][cell_id]['metadata'] = update_city_row_metadata(cell['metadata'])
                merged_json['rows'][row_id]['cells'][cell_id]['annotationMeta'] = {
                    'annotated': True,
                    'match': {
                        'value': True,
                        'reason': 'reconciliator'
                    },
                    'lowestScore': 1,
                    'highestScore': 1
                }

        # Add weather data
        for prop in properties:
            cell_id = f"{reconciliated_column_name}_{prop}"
            merged_json['rows'][row_id]['cells'][cell_id] = {
                'id': f"{row_id}${cell_id}",
                'label': extension_json['rows'][row_id]['cells'][prop]['label'],
                'metadata': []
            }

    return merged_json

def format_json_for_backend(merged_json):
    backend_json = {
        "table": {
            "id": merged_json["table"]["id"],
            "idDataset": merged_json["table"]["idDataset"],
            "name": merged_json["table"]["name"],
            "nCols": merged_json["table"]["nCols"],
            "nRows": merged_json["table"]["nRows"],
            "nCells": merged_json["table"]["nCells"],
            "nCellsReconciliated": merged_json["table"]["nCellsReconciliated"],
            "lastModifiedDate": merged_json["table"]["lastModifiedDate"],
            "minMetaScore": merged_json["table"].get("minMetaScore", 0),
            "maxMetaScore": merged_json["table"].get("maxMetaScore", 0)
        },
        'columns': merged_json['columns'],
        'rows': merged_json['rows'],
        'id': merged_json['id']
    }
    return backend_json

def process_and_format_json(reconciled_json, extension_json, reconciliated_column_name, properties):
    merged_json = merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties)
    formatted_json = format_json_for_backend(merged_json)
    return formatted_json


In [122]:
def update_city_column_metadata(input_metadata):
    if not input_metadata:
        return []
    
    first_meta = input_metadata[0]
    return [{
        'id': first_meta.get('id', 'None:'),
        'match': first_meta.get('match', True),
        'score': first_meta.get('score', 0),
        'name': first_meta.get('name', {'value': '', 'uri': ''}),
        'entity': [
            {
                'id': entity.get('id', ''),
                'name': entity.get('name', {'value': '', 'uri': ''}),
                'score': entity.get('score', 0),
                'match': entity.get('match', True),
                'type': entity.get('type', [])
            }
            for entity in first_meta.get('entity', [])
        ]
    }]

def update_city_row_metadata(input_metadata):
    if not input_metadata:
        return []
    
    first_meta = input_metadata[0]
    return [{
        'id': first_meta.get('id', ''),
        'name': first_meta.get('name', {'value': '', 'uri': ''}),
        'feature': first_meta.get('feature', [{'id': 'all_labels', 'value': 100}]),
        'score': first_meta.get('score', 1),
        'match': first_meta.get('match', True),
        'type': first_meta.get('type', [])
    }]

def merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties):
    merged_json = {
        'table': extension_json['table'],
        'columns': {},
        'rows': {},
        'id': extension_json['id']
    }

    # Process columns
    for col_id, col in reconciled_json['columns'].items():
        merged_json['columns'][col_id] = {
            'id': col['id'],
            'label': col['label'],
            'status': col['status'],
            'context': col['context'],
            'metadata': col['metadata']
        }
        if col_id == reconciliated_column_name:
            merged_json['columns'][col_id]['status'] = 'reconciliated'
            merged_json['columns'][col_id]['metadata'] = update_city_column_metadata(col['metadata'])
            merged_json['columns'][col_id]['annotationMeta'] = {
                'annotated': True,
                'match': {
                    'value': True,
                    'reason': 'reconciliator'
                },
                'lowestScore': 0,
                'highestScore': 0
            }

    # Add weather columns
    for prop in properties:
        col_id = f"{reconciliated_column_name}_{prop}"
        merged_json['columns'][col_id] = {
            'id': col_id,
            'label': col_id,
            'metadata': [],
            'status': 'empty',
            'context': {}
        }

    # Process rows
    for row_id, row in reconciled_json['rows'].items():
        merged_json['rows'][row_id] = {
            'id': row['id'],
            'cells': {}
        }
        for cell_id, cell in row['cells'].items():
            merged_json['rows'][row_id]['cells'][cell_id] = {
                'id': cell['id'],
                'label': cell['label'],
                'metadata': cell.get('metadata', [])
            }
            if cell_id == reconciliated_column_name:
                merged_json['rows'][row_id]['cells'][cell_id]['metadata'] = update_city_row_metadata(cell['metadata'])
                merged_json['rows'][row_id]['cells'][cell_id]['annotationMeta'] = {
                    'annotated': True,
                    'match': {
                        'value': True,
                        'reason': 'reconciliator'
                    },
                    'lowestScore': 1,
                    'highestScore': 1
                }

        # Add weather data
        for prop in properties:
            cell_id = f"{reconciliated_column_name}_{prop}"
            merged_json['rows'][row_id]['cells'][cell_id] = {
                'id': f"{row_id}${cell_id}",
                'label': extension_json['rows'][row_id]['cells'][prop]['label'],
                'metadata': []
            }

    return merged_json

def format_json_for_backend(merged_json):
    backend_json = {
        "table": {
            "id": merged_json["table"]["id"],
            "idDataset": merged_json["table"]["idDataset"],
            "name": merged_json["table"]["name"],
            "nCols": merged_json["table"]["nCols"],
            "nRows": merged_json["table"]["nRows"],
            "nCells": merged_json["table"]["nCells"],
            "nCellsReconciliated": merged_json["table"]["nCellsReconciliated"],
            "lastModifiedDate": merged_json["table"]["lastModifiedDate"],
            "minMetaScore": merged_json["table"].get("minMetaScore", 0),
            "maxMetaScore": merged_json["table"].get("maxMetaScore", 0)
        },
        'columns': merged_json['columns'],
        'rows': merged_json['rows'],
        'id': merged_json['id']
    }
    return backend_json

def process_and_format_json(reconciled_json, extension_json, reconciliated_column_name, properties):
    merged_json = merge_reconciled_and_extension(reconciled_json, extension_json, reconciliated_column_name, properties)
    formatted_json = format_json_for_backend(merged_json)
    return formatted_json


In [123]:
# Load JSON files from drive
def load_json_from_drive(file_path):
    import json
    with open(file_path, 'r') as file:
        return json.load(file)

reconciled_json_path = '/Users/abubakarialidu/Documents/SEMT-py/semtui1.1/semtui_refactored/my_reconciled_table.json'
extension_json_path = '/Users/abubakarialidu/Documents/SEMT-py/semtui1.1/semtui_refactored/my_extended_table.json'

reconciled_json = load_json_from_drive(reconciled_json_path)
extension_json = load_json_from_drive(extension_json_path)

# Example usage
reconciliated_column_name = 'City'  # Column that contains reconciled IDs
properties = ["apparent_temperature_max", "apparent_temperature_min", "precipitation_sum"]  # Replace with actual properties to extend

formatted_json = process_and_format_json(reconciled_json, extension_json, reconciliated_column_name, properties)


In [124]:
formatted_json

{'table': {'id': '148',
  'idDataset': '30',
  'name': 'Jot_test1',
  'nCols': 7,
  'nRows': 3,
  'nCells': 21,
  'nCellsReconciliated': 0,
  'lastModifiedDate': '2024-06-07T14:05:32.939Z',
  'minMetaScore': 0,
  'maxMetaScore': 0},
 'columns': {'Fecha_id': {'id': 'Fecha_id',
   'label': 'Fecha_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Keyword': {'id': 'Keyword',
   'label': 'Keyword',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Impresiones': {'id': 'Impresiones',
   'label': 'Impresiones',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Clicks': {'id': 'Clicks',
   'label': 'Clicks',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'City': {'id': 'City',
   'label': 'City',
   'status': 'reconciliated',
   'context': {'georss': {'uri': 'http://www.google.com/maps/place/',
     'total': 3,
     'reconciliated': 3}},
   'metadata': [{'id': '',
     'match': True,
     'score': 0,
     'name': {'value': '', 'uri': '

In [125]:
import json
import requests

def construct_payload(formatted_json, table_id, id_dataset, table_name):
    table_data = formatted_json['table']
    
    columns_data = formatted_json['columns']
    rows_data = formatted_json['rows']

    # Calculate nCellsReconciliated
    nCellsReconciliated = sum(
        1 for row in rows_data.values() for cell in row['cells'].values() if cell.get('annotationMeta', {}).get('annotated', False)
    )

    # Ensure new columns are included in the payload
    new_columns = ["City_apparent_temperature_max", "City_apparent_temperature_min", "City_precipitation_sum"]
    for new_col in new_columns:
        if new_col not in columns_data:
            columns_data[new_col] = {
                'id': new_col,
                'label': new_col,
                'status': 'empty',
                'context': {},
                'metadata': []
            }

    # Ensure new columns are included in each row's cells
    for row_id, row in rows_data.items():
        for new_col in new_columns:
            if new_col not in row['cells']:
                row['cells'][new_col] = {
                    'id': f"{row_id}${new_col}",
                    'label': '',
                    'metadata': []
                }

    # Construct the payload
    payload = {
        "tableInstance": {
            "id": table_id,
            "idDataset": id_dataset,
            "name": table_name,
            "nCols": table_data["nCols"],
            "nRows": table_data["nRows"],
            "nCells": table_data["nCells"],
            "nCellsReconciliated": nCellsReconciliated,
            "lastModifiedDate": table_data["lastModifiedDate"],
            "minMetaScore": table_data["minMetaScore"],
            "maxMetaScore": table_data["maxMetaScore"]
        },
        "columns": {
            "byId": columns_data,
            "allIds": list(columns_data.keys())
        },
        "rows": {
            "byId": rows_data,
            "allIds": list(rows_data.keys())
        }
    }

    return payload

def push_to_backend(payload, url):
    headers = {'Content-Type': 'application/json'}
    response = requests.put(url, data=json.dumps(payload), headers=headers)
    return response

# Example usage
table_id = '160'
id_dataset = '30'
table_name = 'test_e2'

# Construct the payload
payload = construct_payload(formatted_json, table_id, id_dataset, table_name)

# Log the payload being sent
print("Payload being sent:")
print(json.dumps(payload, indent=2))

# Push to backend
backend_url = f"{api_url}dataset/{dataset_id}/table/{table_id}"
response = push_to_backend(payload, backend_url)

# Log the response
print(f"Status Code: {response.status_code}")
print(f"Response: {response.text}")

# Verify the payload being sent
print("Payload sent:")
print(json.dumps(payload, indent=2))

Payload being sent:
{
  "tableInstance": {
    "id": "160",
    "idDataset": "30",
    "name": "test_e2",
    "nCols": 7,
    "nRows": 3,
    "nCells": 21,
    "nCellsReconciliated": 3,
    "lastModifiedDate": "2024-06-07T14:05:32.939Z",
    "minMetaScore": 0,
    "maxMetaScore": 0
  },
  "columns": {
    "byId": {
      "Fecha_id": {
        "id": "Fecha_id",
        "label": "Fecha_id",
        "status": "empty",
        "context": {},
        "metadata": []
      },
      "Keyword": {
        "id": "Keyword",
        "label": "Keyword",
        "status": "empty",
        "context": {},
        "metadata": []
      },
      "Impresiones": {
        "id": "Impresiones",
        "label": "Impresiones",
        "status": "empty",
        "context": {},
        "metadata": []
      },
      "Clicks": {
        "id": "Clicks",
        "label": "Clicks",
        "status": "empty",
        "context": {},
        "metadata": []
      },
      "City": {
        "id": "City",
        "label": 

In [65]:
def push_reconciliation_data_to_backend(token_manager, dataset_id, table_id, payload, api_url):
    """
    Pushes reconciliation data to the backend.

    :param token_manager: Instance of TokenManager to handle authentication tokens
    :param dataset_id: ID of the dataset
    :param table_id: ID of the table
    :param Reconciled_data: Reconciled data to be sent
    :param api_url: Base URL of the API
    :return: Response from the backend
    """
    # Get the token
    token = token_manager.get_token()
    headers = {
        'Authorization': f'Bearer {token}',
        'Content-Type': 'application/json'
    }

    # Create the endpoint URL
    url = f"{api_url}dataset/{dataset_id}/table/{table_id}"

    # Create the update payload
    update_payload = payload

    try:
        # Send the PUT request to update the table
        response = requests.put(url, headers=headers, json=update_payload)
        
        if response.status_code == 200:
            print("Table updated successfully!")
            return response.json()
        elif response.status_code == 401:
            print("Unauthorized: Invalid or missing token.")
        elif response.status_code == 404:
            print(f"Dataset or table with ID {dataset_id}/{table_id} not found.")
        else:
            print(f"Failed to update table: {response.status_code}, {response.text}")
    except requests.exceptions.RequestException as e:
        print(f"Error occurred while updating table: {e}")
        return None

In [126]:
try:
    table_data = dataset_manager.get_table_by_name(dataset_id, table_name)  # Attempt to retrieve the specified table from the dataset
    if table_data:
        print(f"Table '{table_name}' retrieved successfully!")  # Print success message if table is retrieved
        # No need to display the DataFrame
    else:
        print(f"Table '{table_name}' not found in the dataset.")  # Print message if table is not found
except Exception as e:
    print(f"Error retrieving table '{table_name}': {e}")  # Print error message if retrieving the table fails

Table 'test_e2' retrieved successfully!


In [127]:
table_data

{'table': {'id': '160',
  'idDataset': '30',
  'name': 'test_e2',
  'nCols': 10,
  'nRows': 3,
  'nCells': 21,
  'nCellsReconciliated': 3,
  'lastModifiedDate': '2024-06-21T11:55:47.406Z',
  'minMetaScore': 0,
  'maxMetaScore': 0},
 'columns': {'Fecha_id': {'id': 'Fecha_id',
   'label': 'Fecha_id',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Keyword': {'id': 'Keyword',
   'label': 'Keyword',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Impresiones': {'id': 'Impresiones',
   'label': 'Impresiones',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'Clicks': {'id': 'Clicks',
   'label': 'Clicks',
   'status': 'empty',
   'context': {},
   'metadata': []},
  'City': {'id': 'City',
   'label': 'City',
   'status': 'reconciliated',
   'context': {'georss': {'uri': 'http://www.google.com/maps/place/',
     'total': 3,
     'reconciliated': 3}},
   'metadata': [{'id': '',
     'match': True,
     'score': 0,
     'name': {'value': '', 'uri': ''

In [66]:
reconciliated_column_name = 'City'
table_id = '160'
dataset_id = '30'
table_name = 'test_e2'
push_reconciliation_data_to_backend(token_manager, dataset_id, table_id, payload, api_url)

Table updated successfully!


{'name': 'Jot_test1',
 'nRows': 3,
 'nCols': 10,
 'nCells': 30,
 'nCellsReconciliated': 3,
 'minMetaScore': 0,
 'maxMetaScore': 1,
 'lastModifiedDate': '2024-06-20T14:32:12.742Z'}